In [75]:
import requests
import re
import json
from tqdm.auto import tqdm
from collections import defaultdict
import pickle 
import pandas as pd
import pymorphy2
import time

from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk

from sklearn.metrics.pairwise import linear_kernel

In [76]:
#скачиваем с api вакансии
sber = '3529'
page = 1
num_per_page = 100
ekaterinburg = 1261

url = f'https://api.hh.ru/vacancies?employer_id={sber}&page={page}&per_page={num_per_page}&area={ekaterinburg}'
res = requests.get(url)
vacancies = res.json()
if vacancies is None:
        time.sleep(600)
        
# извлекаем Id вакансий
all_vacancy_ids = []
for i in tqdm(range(vacancies.get('pages'))):
    num_pages = vacancies.get('pages')
    vacancy_id = [el.get('id') for el in vacancies.get('items')]
    all_vacancy_ids.extend(vacancy_id)

  0%|          | 0/3 [00:00<?, ?it/s]

In [77]:
all_vacs= {}
all_vacids = all_vacancy_ids


for vacid in tqdm(all_vacids):
    url = f'https://api.hh.ru/vacancies/{vacid}?host=hh.ru'
    paramz = {
            'employer_id':sber,
            'per_page':num_per_page,
            'area': ekaterinburg,
        }
    res = requests.get(url, params=paramz)
    all_vacs[vacid]= res.json()

  0%|          | 0/300 [00:00<?, ?it/s]

In [78]:
tabl = {'id': [], 'name': [],'description': [], 'alternate_url': []}

In [79]:
for k, v in all_vacs.items():
    tabl['id'].append(k)
    tabl['name'].append(v.get('name'))
    tabl['description'].append(v.get('description'))
    tabl['alternate_url'].append(v.get('alternate_url'))
df = pd.DataFrame(tabl)

In [80]:
#предобрабатываем и добавляем преобразованный текст в новый столбец 'text_clean'
#удаляем пропуски
df.dropna(subset = ['description'], inplace = True)

In [81]:
# функция удаления пунктуации и цифр
def remove_trash(x): 
    pattern = '[^А-Яа-я0-9]+'
    try:
      x = [re.sub(pattern, ' ', i) for i in x] 
      print (x)
    except Exception as e:
      print(e)
    return x
df['text_clean'] = remove_trash(df['description'])

# функция удаления html-тэгов
def remove_trash(x): 
    pattern = r'<.*?>'
    try:
      x = [re.sub(pattern, ' ', i) for i in x] 
      print (x)
    except Exception as e:
      print(e)
    return x
df['text_clean'] = remove_trash(df['description'])

[' 20 занятости Если ты хочешь работать в дружном коллективе стремишься к достижению лучших результатов для клиентов Начни работу в крупнейшем банке России и СНГ Как менеджер по обслуживанию ты будешь осуществлять операции по обслуживанию физических лиц проводить кассовые операции прием выдача обмен денег Ты подходишь нам если ты специалист со средним профессиональным неполным высшим высшим образованием владеешь программами внимателен к деталям готов к работе предполагающей ежедневное проведение однотипных операций проявляешь доброжелательность готовность оказывать помощь окружающим выдерживаешь высокий темп работы Мы предлагаем трудоустройство согласно ТК РФ гарантированный доход плюс премиальное вознаграждение возможность работать рядом с домом местом учебы регулярное корпоративное обучение ДМС страхование от несчастных случаев и тяжелых заболеваний материальную помощь и социальную поддержку корпоративную пенсионную программу льготные условия кредитования яркую и насыщенную корпорати

In [82]:
# функция лемматизации - приводит слова в 1 форму
def lemmatize(row):
    t = []
    text = row['text_clean']
    for word in text.split():
        if len(word)<=3:   #удаляет слова, которые меньше == 3 символа
            continue
        p = morph.parse(word)[0]
        t.append(p.normal_form)
    return " ".join(t)
df['text_clean'] = df.apply(lemmatize,axis=1)

In [83]:
#удалить часто встречающиеся слова и стоп-слова
stop_russian = stopwords.words('russian')

#текст преобразуем в векторное представление
text_transformer = TfidfVectorizer(stop_words=stop_russian, ngram_range=(1,1), lowercase=True, max_features=15000 )
text = text_transformer.fit_transform(df['text_clean'])

In [84]:
#что ищем
text_to_find = 'сбор данных из внутренних и внешних источников, подготовкf данных к анализу, разработкf моделей обработки и анализа данных, в том числе с применением алгоритмов машинного обучения в рамках аудиторских проверок по направлениям аудита корпоративного бизнеса, операций управления благосостоянием и операций на финансовых рынках, взаимодействие с аналитиками, другими командами.'

In [85]:
#создали матрицу по поиску
text_to_find_matrix = text_transformer.transform([text_to_find])

# расчет косинусного расстояния
cosine_similarities = linear_kernel(text, text_to_find_matrix).flatten()

# запишем все попарные результаты сравнений
df['cos_similarities'] = cosine_similarities

# и отсортируем по убыванию (т.к. cos(0)=1)
df = df.sort_values(by=['cos_similarities'], ascending=[0])

In [86]:
#выводим ТОП 10 подходящтх вакансий
df1 = df[['name', 'alternate_url', 'cos_similarities' ]].head(10)
df1

,name,alternate_url,cos_similarities
48,Аналитик данных (клиентская аналитика),https://hh.ru/vacancy/48480378,0.243086
53,Разработчик хранилищ данных,https://hh.ru/vacancy/49073609,0.113257
47,Системный аналитик (Wealth management),https://hh.ru/vacancy/49056195,0.066183
14,Старший инженер по сопровождению,https://hh.ru/vacancy/48039402,0.062208
89,Системный аналитик middle,https://hh.ru/vacancy/48480383,0.059882
29,Руководитель офиса,https://hh.ru/vacancy/49001838,0.056207
95,Android разработчик,https://hh.ru/vacancy/49033339,0.054028
91,DevOps инженер,https://hh.ru/vacancy/49071139,0.050486
27,Бизнес/Системный аналитик,https://hh.ru/vacancy/48389540,0.049961
46,Системный аналитик DWH,https://hh.ru/vacancy/48495851,0.045307
